Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [120]:
import pandas as pd 


In [121]:
ratings = pd.read_csv('ratings.csv')
reviews_grouped = ratings.groupby('userId').agg({'movieId':['count'], 'timestamp':['min','max']}).reset_index()
reviews_grouped['lifetime'] = reviews_grouped['timestamp']['max'] - reviews_grouped['timestamp']['min']
print(f'Среднее время жизни всех пользователей {reviews_grouped.lifetime.mean()/60/60/24:.0f} дней')
mean = reviews_grouped[reviews_grouped['movieId']['count'] > 100].lifetime.mean()/60/60/24
print(f'Среднее время жизни пользователей с более чем 100 оценками {mean:.0f} дней')

Среднее время жизни всех пользователей 201 дней
Среднее время жизни пользователей с более чем 100 оценками 464 дней


Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [122]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [123]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [133]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [134]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [144]:
merged_without_addresses = rzd.merge(auto, how='outer').merge(air, how='outer')
merged_without_addresses

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [145]:
merged_with_addresses = client_base.merge(rzd, how='left').merge(auto, how='left').merge(air,how='left')
merged_with_addresses

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


1). Если я правильно понял условия задачии, у нас есть большое количество последовательностей координат, когда клиент был онлайн. Также есть время, когда тот или иной пользователь находился по соотв. координатам. 
Если всё верно, то необходимо получить время интервал визита или покупки, и найти интервал между этим временем и временем из таблицы последовательностей координат. Для того времени, когда этот интервал будет минимальный, и нужно сохранить координаты - это и будет время покупки или визита. 
2). На расчётах это не должно отразиться. 
3).Можно каким-то образом объединить координаты в районы, и посмотреть, в каких районах(городах, странах, полушариях) нас покупают и просматривают чаще всего. Для этого можно использовать библиотеку geopy. если использовать время, то можно определить, где работают и где живут наши клиенты. 